In [1]:
# Import necessary libraries
import xarray as xr
import cdsapi
import xarray as xr
import rioxarray
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
import os  

In [2]:
# Load India shapefile
india_shapefile = gpd.read_file(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\SHAPEFILE\gadm36_IND_0.shp").to_crs("epsg:4326")
india_geometry = india_shapefile.geometry

In [4]:
# Load Climate data (.nc format)
climate_data = xr.open_dataset(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\CDS\weather_data.nc")